In [ ]:
!pip install pytrends
from pytrends.request import TrendReq
import pandas as pd
from datetime import date
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [ ]:
## daily range
date_range = pd.date_range(start="2020-10-01", end="2020-11-01")

In [ ]:
## import pytrends, final df format
pytrends = TrendReq(hl='en-US', tz=360)
final_df = pd.DataFrame([], columns = ['geoName', 'geoCode', 'value', 'topic', 'date'])

In [ ]:
## keywords
kw_list = ['Transportation', 
           'Domestic Violence']

cat_list = ['Transportation', 'Domestic Violence']

In [ ]:
idx = 0

for i in kw_list:
  ## temporary df format for each topic
  new_df = pd.DataFrame([], columns = ['geoName', 'geoCode', i, 'date'])
  
  for j in range(0, len(date_range)-1):
    tf = '' + date_range[j].strftime('%Y-%m-%d') + ' ' + date_range[j+1].strftime('%Y-%m-%d')
    pytrends.build_payload([i], timeframe = tf, geo='US-GA')
    df = pytrends.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=True)
    df = df.reset_index()

    date = []
    for k in range(0, len(df)):
      date.append(date_range[j].strftime('%Y-%m-%d'))
        
    df['date'] = date
    new_df = new_df.append(df, ignore_index=True)
  

  new_df.rename(columns={i: 'value'}, inplace = True)

  ## add topic column
  cat = cat_list[idx]
  idx += 1
  topic = []
  for l in range(0, len(new_df)):
    topic.append(cat)
  new_df['topic'] = topic
  
  final_df = final_df.append(new_df, ignore_index= True)

ResponseError: ignored

In [ ]:
final_df

,geoName,geoCode,value,topic,date
0,Albany GA,525,58,Transportation,2020-09-01
1,Atlanta GA,524,24,Transportation,2020-09-01
2,Augusta GA,520,29,Transportation,2020-09-01
3,Chattanooga TN,575,20,Transportation,2020-09-01
4,Columbus GA,522,0,Transportation,2020-09-01
...,...,...,...,...,...
595,Dothan AL,606,0,Domestic Violence,2020-09-30
596,Jacksonville FL,561,0,Domestic Violence,2020-09-30
597,Macon GA,503,0,Domestic Violence,2020-09-30
598,Savannah GA,507,0,Domestic Violence,2020-09-30


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## connecting gspreads
scope = [
'https://spreadsheets.google.com/feeds'
]
json_file_name = '/directory_stored_json_file/***.json' ## refers to the README to download your own JSON file
creds = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
client = gspread.authorize(creds)

## open by url
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1y4nR64TtGQ0F9sUvE0hzDN1xh9TZYfb0u9un82TECKY/' # go on link to view dataset
sheet = client.open_by_url(spreadsheet_url)
worksheet = sheet.worksheet('Sheet1')

## update google sheet
existing = get_as_dataframe(worksheet)
existing = existing.dropna(how="all")
existing = existing.iloc[:, 0:5]
update = existing.append(final_df, ignore_index= True)
set_with_dataframe(worksheet, update)